pytorchでは，データセット，モジュール(モデル)，損失，optimizerは別々に考える．

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303898  [    0/60000]
loss: 2.287594  [ 6400/60000]
loss: 2.275912  [12800/60000]
loss: 2.268220  [19200/60000]
loss: 2.252248  [25600/60000]
loss: 2.227209  [32000/60000]
loss: 2.224413  [38400/60000]
loss: 2.197481  [44800/60000]
loss: 2.188936  [51200/60000]
loss: 2.158232  [57600/60000]
Test Error: 
 Accuracy: 48.9%, Avg loss: 2.155688 

Epoch 2
-------------------------------
loss: 2.158772  [    0/60000]
loss: 2.145471  [ 6400/60000]
loss: 2.101736  [12800/60000]
loss: 2.113832  [19200/60000]
loss: 2.058746  [25600/60000]
loss: 2.008996  [32000/60000]
loss: 2.018992  [38400/60000]
loss: 1.947220  [44800/60000]
loss: 1.946721  [51200/60000]
loss: 1.878213  [57600/60000]
Test Error: 
 Accuracy: 59.2%, Avg loss: 1.880523 

Epoch 3
-------------------------------
loss: 1.903885  [    0/60000]
loss: 1.871547  [ 6400/60000]
loss: 1.771024  [12800/60000]
loss: 1.806234  [19200/60000]
loss: 1.694304  [25600/60000]
loss: 1.651182  [32000/600